In [1]:
# pip install selenium
# pip install easyocr

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time

In [9]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 드라이버 생성
driver = webdriver.Chrome(options=chrome_options)

# 브라우저 사이즈
driver.set_window_size(1900,1000)

# 웹페이지가 로드될때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)

driver.get(url='https://tickets.interpark.com/')

In [10]:
import os

# personal.txt 파일에서 아이디와 비밀번호 읽어오기
def get_credentials(file_path):
    personal = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            personal[key] = value
    return personal

# personal.txt 파일 경로
file_path = 'personal.txt'

# 아이디와 비밀번호 가져오기
personal = get_credentials(file_path)
username = personal.get('username')
password = personal.get('password')

# 가져온 아이디와 비밀번호를 사용하여 작업 수행
# print(f"Username: {username}")
# print(f"Password: {password}")

# 여기서 WebDriver 실행 등 필요한 작업을 수행합니다.


In [11]:
# 로그인
driver.find_element(By.LINK_TEXT,'로그인').click()
userId = driver.find_element(By.ID, 'userId')
userId.send_keys(username)
userPwd = driver.find_element(By.ID, "userPwd")
userPwd.send_keys(password)
userPwd.send_keys(Keys.ENTER)
# 로그인버튼 클릭
# driver.find_element(By.ID, "btn_login").click()

In [12]:
# 사이트 이동 - 현재 서울이랜드
driver.get('https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07002&TeamCode=PS098')
# 키움 = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07001&TeamCode=PB003
# 두산 = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07001&TeamCode=PB004
# LCK = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07032&TeamCode=PE015
# 서울이랜드 = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07002&TeamCode=PS098

# 첫번째 경기 예매하기 버튼 클릭 CSS_SELECTOR
driver.find_element(By.CSS_SELECTOR, f".timeScheduleList .timeSchedule:nth-of-type({1}) .BtnColor_Y.btn1").click()

In [7]:
import easyocr

In [13]:
# 팝업 창으로 포커싱 이동
driver.switch_to.window(driver.window_handles[1]) # 기존에 열려있던 창이 0, 팝업창이 1
time.sleep(0.5)
# 대기순서 창이 나올 수 있음

In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:
# 보안문자 iframe 창이 나오는 대기순서 class="wrap" 기다리기
wait = WebDriverWait(driver, 100)
wait.until(EC.presence_of_element_located((By.ID, "ifrmSeat")))

# ifrmSeat iframe으로 전환
ifrmSeat = driver.find_element(By.XPATH, '//iframe[@id="ifrmSeat"]')
driver.switch_to.frame(ifrmSeat)
time.sleep(0.5)

In [ ]:
# easyocr 이미지 내에 인식할 언어 지정
reader = easyocr.Reader(['en'])

# 보안문자 입력 함수
def process_captcha():
    # 보안 문자 이미지 캡쳐
    capchaPng = driver.find_element(By.XPATH, "//*[@id='imgCaptcha']")

    # 캡쳐한 이미지에서 문자열 인식하기
    result = reader.readtext(captchaPng.screenshot_as_png, detail=0)

    # 이미지에 점과 직선이 포함되어 있어서 문자 인식이 완벽하지 않아 데이터를 수동으로 보정해줌
    capchaValue = result[0].replace(' ','').replace('5','S').replace('0','O')